In [55]:
!nvidia-smi

Mon Aug  8 09:50:43 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 515.43.04    Driver Version: 515.43.04    CUDA Version: 11.7     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA GeForce ...  On   | 00000000:01:00.0  On |                  N/A |
| N/A   43C    P5     8W /  N/A |   2027MiB /  4096MiB |     39%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [56]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt 
import torch 
from torchvision import datasets , transforms , models 
from torch.utils.data.sampler import SubsetRandomSampler 
import torch.nn as nn 
import torch.nn.functional as F 
from datetime import datetime 

In [57]:
transform = transforms.Compose( 
    [ 
        transforms.Resize(255) , transforms.CenterCrop(224) , transforms.ToTensor() ,
    ]
)

In [58]:
dataset = datasets.ImageFolder("./data/New Plant Diseases Dataset(Augmented)/New Plant Diseases Dataset(Augmented)/train/" , transform=transform)

In [59]:
dataset

Dataset ImageFolder
    Number of datapoints: 70295
    Root location: ./data/New Plant Diseases Dataset(Augmented)/New Plant Diseases Dataset(Augmented)/train/
    StandardTransform
Transform: Compose(
               Resize(size=255, interpolation=bilinear, max_size=None, antialias=None)
               CenterCrop(size=(224, 224))
               ToTensor()
           )

In [60]:
indices = list(range(len(dataset)))
len(indices)

70295

In [61]:
split = int(np.floor(0.85*len(dataset)))
validation = int(np.floor(0.70*split)) 
print(0 , validation , split , len(dataset))

0 41825 59750 70295


In [62]:
print(f"length of train size :{validation}")
print(f"length of validation size :{split - validation}")
print(f"length of test size :{len(dataset)-validation}")

length of train size :41825
length of validation size :17925
length of test size :28470


In [63]:
np.random.shuffle(indices)

In [64]:
train_indices , validation_indices , test_indices = ( 
    indices[:validation] , indices[validation:split] , indices[split:]
)

In [65]:
train_sampler = SubsetRandomSampler(train_indices) 
validation_sampler = SubsetRandomSampler(validation_indices) 
test_sampler = SubsetRandomSampler(test_indices)

In [66]:
targets_size = len(dataset.class_to_idx)

In [67]:
targets_size

38

# Convolution Aithmetic Equation : (W - F + 2P) / S + 1
W = Input Size

F = Filter Size

P = Padding Size

S = Stride

In [68]:
class CNN(nn.Module): 
    def __init__(self , K): 
        super(CNN , self).__init__() 
        self.conv_layers = nn.Sequential( 
            
            #conv1 
            nn.Conv2d(in_channels=3 , out_channels=32  , kernel_size=3 , padding=1),
            nn.ReLU() ,
            nn.BatchNorm2d(32) , 
            nn.Conv2d(in_channels=32 , out_channels=32 , kernel_size=3 , padding=1),
            nn.ReLU() ,
            nn.BatchNorm2d(32) , 
            nn.MaxPool2d(kernel_size=2 , stride=2) , 

            #conv2 
            nn.Conv2d(in_channels=32 , out_channels=64 , kernel_size=3 , padding=1),
            nn.ReLU() ,
            nn.BatchNorm2d(64) , 
            nn.Conv2d(in_channels=64 , out_channels=64 , kernel_size=3 , padding=1),
            nn.ReLU() ,
            nn.BatchNorm2d(64) , 
            nn.MaxPool2d(kernel_size=2 , stride=2) , 

            #conv3 
            nn.Conv2d(in_channels=64 , out_channels=128 , kernel_size=3 , padding=1),
            nn.ReLU() ,
            nn.BatchNorm2d(128) , 
            nn.Conv2d(in_channels=128 , out_channels=128 , kernel_size=3 , padding=1),
            nn.ReLU() ,
            nn.BatchNorm2d(128) , 
            nn.MaxPool2d(kernel_size=2 , stride=2) , 
            
            #conv4 
            nn.Conv2d(in_channels=128 , out_channels=256 , kernel_size=3 , padding=1),
            nn.ReLU() ,
            nn.BatchNorm2d(256) , 
            nn.Conv2d(in_channels=256 , out_channels=256 , kernel_size=3 , padding=1),
            nn.ReLU() ,
            nn.BatchNorm2d(256) , 
            nn.MaxPool2d(kernel_size=2 , stride=2) , 
        )

        self.dense_layers = nn.Sequential( 
            nn.Dropout(0.4), 
            nn.Linear(50176 , 1024) ,
            nn.ReLU() ,
            nn.Dropout(0.4), 
            nn.Linear(1024 , K) ,
        )

    def forward(self , X): 
        out = self.conv_layers(X) 

        # Flatten 
        out = out.view(-1 , 50176) 

        # fully connected 
        out = self.dense_layers(out) 
        return out

In [69]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# device = "cpu"
print(device)

cuda


In [70]:
model = CNN(targets_size)

In [71]:
model.to(device)

CNN(
  (conv_layers): Sequential(
    (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (3): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (4): ReLU()
    (5): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (6): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (7): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU()
    (9): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (10): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU()
    (12): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (13): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (14): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)

In [72]:
from torchsummary import summary

summary(model, (3, 224, 224))

Layer (type:depth-idx)                   Output Shape              Param #
├─Sequential: 1-1                        [-1, 256, 14, 14]         --
|    └─Conv2d: 2-1                       [-1, 32, 224, 224]        896
|    └─ReLU: 2-2                         [-1, 32, 224, 224]        --
|    └─BatchNorm2d: 2-3                  [-1, 32, 224, 224]        64
|    └─Conv2d: 2-4                       [-1, 32, 224, 224]        9,248
|    └─ReLU: 2-5                         [-1, 32, 224, 224]        --
|    └─BatchNorm2d: 2-6                  [-1, 32, 224, 224]        64
|    └─MaxPool2d: 2-7                    [-1, 32, 112, 112]        --
|    └─Conv2d: 2-8                       [-1, 64, 112, 112]        18,496
|    └─ReLU: 2-9                         [-1, 64, 112, 112]        --
|    └─BatchNorm2d: 2-10                 [-1, 64, 112, 112]        128
|    └─Conv2d: 2-11                      [-1, 64, 112, 112]        36,928
|    └─ReLU: 2-12                        [-1, 64, 112, 112]        --
| 

Layer (type:depth-idx)                   Output Shape              Param #
├─Sequential: 1-1                        [-1, 256, 14, 14]         --
|    └─Conv2d: 2-1                       [-1, 32, 224, 224]        896
|    └─ReLU: 2-2                         [-1, 32, 224, 224]        --
|    └─BatchNorm2d: 2-3                  [-1, 32, 224, 224]        64
|    └─Conv2d: 2-4                       [-1, 32, 224, 224]        9,248
|    └─ReLU: 2-5                         [-1, 32, 224, 224]        --
|    └─BatchNorm2d: 2-6                  [-1, 32, 224, 224]        64
|    └─MaxPool2d: 2-7                    [-1, 32, 112, 112]        --
|    └─Conv2d: 2-8                       [-1, 64, 112, 112]        18,496
|    └─ReLU: 2-9                         [-1, 64, 112, 112]        --
|    └─BatchNorm2d: 2-10                 [-1, 64, 112, 112]        128
|    └─Conv2d: 2-11                      [-1, 64, 112, 112]        36,928
|    └─ReLU: 2-12                        [-1, 64, 112, 112]        --
| 

In [73]:
criterion = nn.CrossEntropyLoss()  # this include softmax + cross entropy loss
optimizer = torch.optim.Adam(model.parameters())

# batch gradient descent

In [74]:
def batch_gd(model , criterion , train_loader , validation_loader , epochs): 
    train_losses = np.zeros(epochs) 
    test_losses = np.zeros(epochs) 

    for e in range(epochs): 
        t0 = datetime.now() 
        train_loss = [] 
        for inputs ,targets in train_loader:
            inputs , targets = inputs.to(device) , targets.to(device) 
            optimizer.zero_grad() 
            outputs = model(inputs) 
            loss = criterion(outputs , targets) 
            train_loss.append(loss.item()) 
            loss.backward()  
            optimizer.step() 
        
        train_loss = np.mean(train_loss) 
        validation_loss = [] 

        for inputs , targets in validation_loader:  
            inputs , targets = inputs.to(device) , targets.to(device) 
            outputs = model(inputs) 
            loss = criterion(outputs , targets) 
            validation_loss.append(loss.item()) 
        validation_loss = np.mean(validation_loss) 
        train_losses[e] = train_loss 
        test_losses[e] = validation_loss 

        dt = datetime.now() - t0 
        print(f"Epoch {e+1}/{epochs} , train loss {train_loss:.3f} , test loss {validation_loss:.3f} , time {dt}")
    return train_losses , test_losses


In [75]:
batch_size = 8
train_loader = torch.utils.data.DataLoader( 
    dataset , batch_size = batch_size , sampler = train_sampler 
)
test_loader = torch.utils.data.DataLoader(
    dataset , batch_size = batch_size , sampler = test_sampler
) 
validation_loader = torch.utils.data.DataLoader(
    dataset , batch_size = batch_size , sampler = validation_sampler
)

In [76]:
train_losses, validation_losses = batch_gd(
    model, criterion, train_loader, validation_loader, 5
)

Epoch 1/5 , train loss 3.899 , test loss 3.645 , time 0:13:56.451988
Epoch 2/5 , train loss 3.619 , test loss 3.669 , time 0:14:57.910047
Epoch 3/5 , train loss 3.555 , test loss 3.612 , time 0:15:19.290218
Epoch 4/5 , train loss 3.533 , test loss 3.467 , time 0:15:05.269963
Epoch 5/5 , train loss 3.486 , test loss 3.537 , time 0:15:38.562884


#### 1- Underfits, when the training loss is way more significant than the testing loss.
#### 2- Overfits, when the training loss is way smaller than the testing loss.

In [77]:
# save the model
torch.save(model.state_dict(), 'plant_disease_model.pt')

In [78]:
train_losses, validation_losses

(array([3.89898883, 3.6188631 , 3.5554541 , 3.53257461, 3.4855126 ]),
 array([3.6450488 , 3.66938249, 3.6116027 , 3.46682437, 3.53723439]))

### ==================================== prediction ====================================

In [8]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt 
import torch 
from torchvision import datasets , transforms , models 
from torch.utils.data.sampler import SubsetRandomSampler 
import torch.nn as nn 
import torch.nn.functional as F 
import torchvision.transforms.functional as TF
from datetime import datetime 
from PIL import Image 

In [4]:
class CNN(nn.Module): 
    def __init__(self , K): 
        super(CNN , self).__init__() 
        self.conv_layers = nn.Sequential( 
            
            #conv1 
            nn.Conv2d(in_channels=3 , out_channels=32  , kernel_size=3 , padding=1),
            nn.ReLU() ,
            nn.BatchNorm2d(32) , 
            nn.Conv2d(in_channels=32 , out_channels=32 , kernel_size=3 , padding=1),
            nn.ReLU() ,
            nn.BatchNorm2d(32) , 
            nn.MaxPool2d(kernel_size=2 , stride=2) , 

            #conv2 
            nn.Conv2d(in_channels=32 , out_channels=64 , kernel_size=3 , padding=1),
            nn.ReLU() ,
            nn.BatchNorm2d(64) , 
            nn.Conv2d(in_channels=64 , out_channels=64 , kernel_size=3 , padding=1),
            nn.ReLU() ,
            nn.BatchNorm2d(64) , 
            nn.MaxPool2d(kernel_size=2 , stride=2) , 

            #conv3 
            nn.Conv2d(in_channels=64 , out_channels=128 , kernel_size=3 , padding=1),
            nn.ReLU() ,
            nn.BatchNorm2d(128) , 
            nn.Conv2d(in_channels=128 , out_channels=128 , kernel_size=3 , padding=1),
            nn.ReLU() ,
            nn.BatchNorm2d(128) , 
            nn.MaxPool2d(kernel_size=2 , stride=2) , 
            
            #conv4 
            nn.Conv2d(in_channels=128 , out_channels=256 , kernel_size=3 , padding=1),
            nn.ReLU() ,
            nn.BatchNorm2d(256) , 
            nn.Conv2d(in_channels=256 , out_channels=256 , kernel_size=3 , padding=1),
            nn.ReLU() ,
            nn.BatchNorm2d(256) , 
            nn.MaxPool2d(kernel_size=2 , stride=2) , 
        )

        self.dense_layers = nn.Sequential( 
            nn.Dropout(0.4), 
            nn.Linear(50176 , 1024) ,
            nn.ReLU() ,
            nn.Dropout(0.4), 
            nn.Linear(1024 , K) ,
        )

    def forward(self , X): 
        out = self.conv_layers(X) 

        # Flatten 
        out = out.view(-1 , 50176) 

        # fully connected 
        out = self.dense_layers(out) 
        return out

In [26]:

idx_to_classes = {0: 'Apple___Apple_scab',
                  1: 'Apple___Black_rot',
                  2: 'Apple___Cedar_apple_rust',
                  3: 'Apple___healthy',
                  4: 'Background_without_leaves',
                  5: 'Blueberry___healthy',
                  6: 'Cherry___Powdery_mildew',
                  7: 'Cherry___healthy',
                  8: 'Corn___Cercospora_leaf_spot Gray_leaf_spot',
                  9: 'Corn___Common_rust',
                  10: 'Corn___Northern_Leaf_Blight',
                  11: 'Corn___healthy',
                  12: 'Grape___Black_rot',
                  13: 'Grape___Esca_(Black_Measles)',
                  14: 'Grape___Leaf_blight_(Isariopsis_Leaf_Spot)',
                  15: 'Grape___healthy',
                  16: 'Orange___Haunglongbing_(Citrus_greening)',
                  17: 'Peach___Bacterial_spot',
                  18: 'Peach___healthy',
                  19: 'Pepper,_bell___Bacterial_spot',
                  20: 'Pepper,_bell___healthy',
                  21: 'Potato___Early_blight',
                  22: 'Potato___Late_blight',
                  23: 'Potato___healthy',
                  24: 'Raspberry___healthy',
                  25: 'Soybean___healthy',
                  26: 'Squash___Powdery_mildew',
                  27: 'Strawberry___Leaf_scorch',
                  28: 'Strawberry___healthy',
                  29: 'Tomato___Bacterial_spot',
                  30: 'Tomato___Early_blight',
                  31: 'Tomato___Late_blight',
                  32: 'Tomato___Leaf_Mold',
                  33: 'Tomato___Septoria_leaf_spot',
                  34: 'Tomato___Spider_mites Two-spotted_spider_mite',
                  35: 'Tomato___Target_Spot',
                  36: 'Tomato___Tomato_Yellow_Leaf_Curl_Virus',
                  37: 'Tomato___Tomato_mosaic_virus',
}

In [86]:
def predict(image_obj , idx_to_classes): 
    INPUT_DIM = 224 

    preprocess = transforms.Compose([ 
        transforms.Resize(INPUT_DIM) , 
        transforms.CenterCrop(INPUT_DIM) , 
        transforms.ToTensor() , 
        transforms.Normalize(mean=[0.485, 0.456, 0.406] , std=[0.229, 0.224, 0.225]) ,
    ]) 

    pretrained_model = CNN(38) 
    pretrained_model.load_state_dict( 
        torch.load('plant_disease_model.pt' , map_location=torch.device('cuda'))  
    ) 

    im = image_obj 
    im_preprocessed = preprocess(im) 
    batch_img_tensor = torch.unsqueeze(im_preprocessed , 0) 
    print(type(batch_img_tensor))
    output = pretrained_model(batch_img_tensor)
    output = output.detach().numpy()  
    index = np.argmax(output) 
    predicted_class = idx_to_classes[index] 
    confidence = np.max(output[0])  
    return predicted_class , confidence*100

In [87]:
file_input = './data/test/test/CornCommonRust1.JPG'
im = Image.open(file_input)
print(type(im))
result = predict(im , idx_to_classes)
print(f"predicted class : {result[0]} \n confidence : {result[1]}")

<class 'PIL.JpegImagePlugin.JpegImageFile'>
<class 'torch.Tensor'>
predicted class : Corn___Cercospora_leaf_spot Gray_leaf_spot 
 confidence : 447.4454879760742


In [83]:
def prediction(image_path ,idx_to_classes): 

    model = CNN(38)    
    model.load_state_dict(torch.load("plant_disease_model.pt"))
    model.eval()

    image = Image.open(image_path) 
    image = image.resize((224, 224)) 
    input_data = TF.to_tensor(image) 
    input_data = input_data.view(-1 , 3 , 224 , 224) 
    output = model(input_data)  
    output = output.detach().numpy()  
    index = np.argmax(output) 
    predicted_class = idx_to_classes[index] 
    confidence = np.max(output[0])  
    return predicted_class , confidence*100

In [84]:
file_input = './data/test/test/CornCommonRust1.JPG'
result = prediction(file_input , idx_to_classes)
print(f"predicted class : {result[0]} \n confidence : {result[1]}")

predicted class : Corn___Cercospora_leaf_spot Gray_leaf_spot 
 confidence : -445.97320556640625
